Quick analysis

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.io.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf

In [ ]:
dg = DataGroup('cage.json', load=True)
run = 44
str_query = f'run=={run} and skip==False'
dg.file_keys.query(str_query, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(dg.file_keys[view_cols])

In [ ]:
#get runtime, startime, runtype

runtype_list = np.array(dg.file_keys['runtype'])
runtype = runtype_list[0]

rt_min = dg.file_keys['runtime'].sum()
u_start = dg.file_keys.iloc[0]['startTime']
t_start = pd.to_datetime(u_start, unit='s') # str

In [ ]:
# get scan position

if runtype == 'alp':
    alphaDB = pd.read_hdf('alphaDB.h5')
    scan_pos = alphaDB.loc[alphaDB['run']==run]
    radius = np.array(scan_pos['radius'])[0]
    angle = np.array(scan_pos['angle'])[0]
    angle_det = 270 + angle
    print(f'Radius: {radius}; Angle: {angle}')
    
else:
    radius = 'n/a'
    angle = 'n/a'
    angle_det = 'n/a'
#     print('Not an alpha scan!')
#     continue

In [ ]:
lh5_dir = dg.lh5_user_dir #if user else dg.lh5_dir
print(lh5_dir)
hit_list = lh5_dir + dg.file_keys['hit_path'] + '/' + dg.file_keys['hit_file']
print(hit_list)
df_hit = lh5.load_dfs(hit_list, ['trapEmax', 'bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max', 'tp_0', 'tp_10', 'tp_50', 'tp_80', 'tp_90', 'tp_max'], 'ORSIS3302DecoderForEnergy/hit')
# df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
# print(df_hit.columns)

# dsc = df_hit[['bl','bl_sig','A_10','AoE', 'ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# dsc = df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# print(dsc)

In [ ]:
# Baseline vs E

etype = 'trapEmax'

elo, ehi, epb = 0, 50, 1
blo, bhi, bpb = 0, 10000, 100
nbx = int((ehi-elo)/epb)
nby = int((bhi-blo)/bpb)

h = plt.hist2d(df_hit['trapEmax'], df_hit['bl'], bins=[nbx,nby],
                       range=[[elo, ehi], [blo, bhi]], cmap='jet')

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('bl', ha='right', y=1)
plt.title('Baseline, alpha')
# plt.tight_layout()
plt.show()

In [ ]:
blo, bhi, bpb = 0, 10000, 100
nbx = int((bhi-blo)/bpb)

bl_hist, bins = np.histogram(df_hit['bl'], bins=nbx,
                range=[blo, bhi])

plt.semilogy(bins[1:], bl_hist, ds='steps', c='b', lw=1)

bl_cut_lo, bl_cut_hi = 8500,10000

plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

In [ ]:
# use baseline cut
df_cut = df_hit.query('bl > 8500 and bl < 10000').copy()

#creat new DCR
const = 0.0555
df_cut['dcr_linoff'] = df_cut['dcr_raw'] + const*df_cut['trapEmax']

#create 10-90
df_cut['10-90'] = df_cut['tp_90']- df_cut['tp_10']

#create 50-90
df_cut['50-90'] = df_cut['tp_90']- df_cut['tp_50']

#create 50-80
df_cut['50-80'] = df_cut['tp_80']- df_cut['tp_50']

#create 0-100
df_cut['0-100'] = df_cut['tp_max']- df_cut['tp_0']

#create 50-100
df_cut['50-100'] = df_cut['tp_max']- df_cut['tp_50']

#create 80-100
df_cut['80-100'] = df_cut['tp_max']- df_cut['tp_80']

#create 0-50
df_cut['0-50'] = df_cut['tp_50']- df_cut['tp_0']

In [ ]:
# Make energy spectrum
elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(df_cut['trapEmax'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
# plt.legend()
plt.title(f'{etype}: {runtype} run {run}; {rt_min:.2f} mins')
# plt.savefig(f'./plots/normScan/{runtype}_energy_run{run}.pdf', dpi=200)
plt.show()

In [ ]:
# 0-50
tlo, thi, tpb = 0, 5000, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['0-50'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('0-50', ha='right', y=1)
# plt.legend()
plt.title(f'0-50: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 0-50 1D hist

tlo, thi, tpb = -500, 50000, 2000

nbx = int((thi-tlo)/tpb)

t50_hist, bins = np.histogram(df_cut['0-50'], bins=nbx,
                range=[tlo, thi])

plt.semilogy(bins[1:], t50_hist, ds='steps', c='b', lw=1) #, label=f'{etype}'



# plt.legend()
plt.xlabel('tp 0-50 (ns)', ha='right', y=1)
plt.ylabel('cts', ha='right', y=1)
plt.title(f'tp 0-50: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_1d_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs 0-50


dlo, dhi, dpb = -100, 200, 0.6
tlo, thi, tpb = -0, 700, 10

nbx = int((dhi-dlo)/dpb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['dcr_linoff'], df_cut['0-50'], bins=[nbx,nby],
                range=[[dlo, dhi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('DCR (arb)', ha='right', x=1)
plt.ylabel('tp 0-50 (ns)', ha='right', y=1)
plt.title(f'DCR vs 0-50% rise time: \n Radius: {radius} mm; Angle {angle_det} deg \n {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 10-90
tlo, thi, tpb = 0, 600, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['10-90'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('10-90', ha='right', y=1)
# plt.legend()
plt.title(f'10-90: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 50-90
tlo, thi, tpb = 0, 600, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['50-90'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('50-90', ha='right', y=1)
# plt.legend()
plt.title(f'50-90: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 50-80
tlo, thi, tpb = 0, 600, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['50-80'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('50-80', ha='right', y=1)
# plt.legend()
plt.title(f'50-80: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 0-100
tlo, thi, tpb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['0-100'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('0-100', ha='right', y=1)
# plt.legend()
plt.title(f'0-100: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 50-100
tlo, thi, tpb = 100, 1500, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['50-100'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('50-100', ha='right', y=1)
# plt.legend()
plt.title(f'50-100: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# 80-100
tlo, thi, tpb = 100, 1500, 10
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((thi-tlo)/tpb)

h = plt.hist2d(df_cut['trapEmax'], df_cut['80-100'], bins=[nbx,nby],
                range=[[elo, ehi], [tlo, thi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('80-100', ha='right', y=1)
# plt.legend()
plt.title(f'80-100: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_run{run}.png', dpi=200)
plt.show()

In [ ]:
# A/E vs 10-90

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = 100, 1000, 10

nbx = int((ahi-alo)/apb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['10-90'], bins=[nbx,nby],
                range=[[alo, ahi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('10-90', ha='right', y=1)
plt.title(f'A/e vs 10-90: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# A/E vs 0-100

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = 400, 10000, 10

nbx = int((ahi-alo)/apb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['0-100'], bins=[nbx,nby],
                range=[[alo, ahi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('0-100', ha='right', y=1)
plt.title(f'A/e vs 0-100: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# A/E vs 50-90

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = 0, 500, 10

nbx = int((ahi-alo)/apb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['50-90'], bins=[nbx,nby],
                range=[[alo, ahi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('50-90', ha='right', y=1)
plt.title(f'A/e vs 50-90: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# A/E vs 50-80

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = 0, 1000, 10
# tlo, thi, tpb = 0, 50000, 10

nbx = int((ahi-alo)/apb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['50-80'], bins=[nbx,nby],
                range=[[alo, ahi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('50-80', ha='right', y=1)
plt.title(f'A/e vs 50-80: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# A/E vs 50-100

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = 0, 4000, 10
# tlo, thi, tpb = 0, 40000, 10

nbx = int((ahi-alo)/apb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['50-100'], bins=[nbx,nby],
                range=[[alo, ahi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('50-100', ha='right', y=1)
plt.title(f'A/e vs 50-100: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# A/E vs 80-100

alo, ahi, apb = 0.0, 0.09, 0.0001
# dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = -100, 40000, 10

nbx = int((ahi-alo)/apb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['AoE'], df_cut['80-100'], bins=[nbx,nby],
                range=[[alo, ahi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('80-100', ha='right', y=1)
plt.title(f'A/e vs 80-100: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()

In [ ]:
# DCR vs 0-50


dlo, dhi, dpb = -100, 300, 0.6
tlo, thi, tpb = -100, 50000, 10

nbx = int((dhi-dlo)/dpb)
nby = int((thi-tlo)/tpb)

alpha_dcr_hist = plt.hist2d(df_cut['dcr_linoff'], df_cut['0-50'], bins=[nbx,nby],
                range=[[dlo, dhi], [tlo, thi]], cmap='jet', norm=LogNorm()) #

plt.xlabel('DCR', ha='right', x=1)
plt.ylabel('0-50', ha='right', y=1)
plt.title(f'DCR vs 0-50: {runtype} run {run}, {rt_min:.2f} mins')
plt.tight_layout()
# plt.savefig(f'./plots/normScan/{runtype}_a10oE_vs_dcr_linoffset_run{run}.png', dpi=200)
plt.show()